In [ ]:
#ライブラリの読み込み
import pandas as pd
from copy import deepcopy

#自作関数の読み込み
import modules.form_to_dict as ftd
import modules.shift_calculation_engine as sce
import modules.shift_visualization as sv
import modules.required_employees_culculation as rec

In [ ]:
# フォームの回答結果から従業員名(list)と出勤可否(dict)を作成
employees_name, availablity_dict_origin = ftd.form_of_employees_availability_to_dict("data/availability_test.csv")

In [ ]:
# データのカラムは下で定義
data = [
    ["6F", "East", 20, "19:30", "22:30"],
    ["5F", "West", 20, "13:00", "17:00"],
    ["4F", "Terrace", 20, "8:30", "13:30"],
]

In [ ]:
required_df = rec.required_employees(data)

# 各フロアごとの従業員数
employee_4f = required_df.loc["4F"].to_list()
employee_5f = required_df.loc["5F"].to_list()
employee_6f = required_df.loc["6F"].to_list()

In [ ]:
floors_dict = {"4F": employee_4f, "5F": employee_5f, "6F": employee_6f}

# 時間を30分ごとのスロットに変換する関数
def time_to_slot(time):
    return int(time * 2) - 14

# シフトにアサインされた時間を出勤不可に更新する関数
def update_availability(shifts, availability):
    for name, times in shifts.items():
        if name in availability:
            start_slot = time_to_slot(times[0])
            end_slot = time_to_slot(times[1])
            for i in range(start_slot, end_slot):
                availability[name][i] = 0
                
    return availability

availability_dict = deepcopy(availablity_dict_origin)

for floor, employee_data in floors_dict.items():
    
    print(availability_dict)
    
    # シフトアサインの計算
    shifts = sce.shift_calculation_engine(employees_name, availability_dict, employee_data)
    
    # 計算されたアサインを表示
    print(shifts)
    
    # シフトの可視化（タイトルにフロア名を使用）
    sv.shift_visualization(shifts, floor)

    # 出勤可否を更新
    availability_dict = update_availability(shifts, availability_dict)

### 次やること
・1日に対する各会場への従業員割当は完了したので、その割り当ての効用を最大化するようにしたい。  
・4階、5階、6階について、現在はそれぞれを順番に計算してシフトを割り当てているので、必然的に4階のアサインが充実して、6階のアサインが乏しくなる。  
・4,5,6階のすべての順番(3!通り)でアルゴリズムを回し、出勤可否のデータに対して損失関数を設定。一番損失が小さいものをシフトの結果とできるような仕組みを作りたい。

## 宴会データのcsvファイルをjson化

In [ ]:
# 宴会婚礼情報のcsvファイルをjson化


parties = pd.read_csv("data/parties_data.csv",encoding='cp932')

# 階層構造に基づいてデータを整理
grouped = parties.groupby(['日程', '会場', 'time'])
structured_data = grouped.apply(lambda x: x.drop(['日程', '会場', 'time'], axis=1).to_dict(orient='records')).reset_index().rename(columns={0: '詳細'})

# JSON形式に変換してファイルに保存
json_data = structured_data.to_json(orient='records', force_ascii=False)
#with open('data.json', 'w', encoding='utf-8') as f:
    #f.write(json_data)

# 結果を確認
print(json_data)
structured_data.head(50)